<a href="https://colab.research.google.com/github/N-aksif-N/MineColab_Improved/blob/free-config/MineColabImproved.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p>
  <img style="display: block; margin-left: auto; margin-right: auto;" src="https://raw.githubusercontent.com/N-aksif-N/MineColab/master/minecolab.png" alt="" width="170" height="136" />
</p>

<h1 style="text-align: center;">
  <span style="color: #00ffff;">MineColab</span>
</h1>

<hr />
  <h2 style="text-align: center;">
    <span style="color: #99ccff;">Forked By N-Aksif</span>
  </h2>

  <h2 style="text-align: center;">
    <span style="color: #FFFFFF;">Run Minecraft Server On Google Colab</span>
  </h2>
  </span>
  </h2>
  <a href="https://github.com/N-aksif-N/MineColab/tree/main" target="_parent"><img align="right" alt="Open In Github"></a>

>[🔥 Starting](#scrollTo=Ei7_3ODcUdm_)

>[▶ or 🛑  Console](#scrollTo=JMgdsuPcUuEv)

>[⚓ Options](#scrollTo=HD4lBUT9RcS-)

>[📎  Log](#scrollTo=hfUxmQTgUF8Z)

>[📰  Software](#scrollTo=JkGq3KakW-Bc)

>[🎈  Plugins, mods](#scrollTo=QaNRBRuK8seq)

>[📁 File Management](#scrollTo=F7efc3oOqTVQ)



----

In [ ]:
# @title **[❗]  Set up** {display-mode: "form"}
# @markdown Check out [wiki of this project](https://github.com/N-aksif-N/MineColab_Improved/wiki) for more explanations

from requests import get
from time import sleep
from json import load, dump
from os import environ
from os.path import exists
try: from jproperties import Properties
except:
  %pip install -q jproperties
  from jproperties import Properties
try: from rich import print
except:
  %pip install -q rich
  from rich import print
if exists('/content/drive') == False:
  from google.colab import drive
  drive.mount('/content/drive')
try: from pyngrok import conf, ngrok
except:
  %pip install -q pyngrok
  from pyngrok import conf, ngrok
try: from bs4 import BeautifulSoup
except:
  %pip install -q BeautifulSoup4
  from bs4 import BeautifulSoup
drive_path = '/content/drive/MyDrive/minecraft'
#------------------------------------------------------------------------------------------------------------------------------------#
def LOG(*args, sep=''):
  check = False
  args = list(args)
  for i in range(len(args)):
    if isinstance(args[i], str):
      if '\n' in args[i]: args[i] = args[i].replace('\n', ''); args.insert(0, '\n[bold green][ LOG ][/bold green] '); check = True; break
  if check != True: args.insert(0, '[bold green][ LOG ][/bold green] ')
  print(sep.join(map(str, args)))
def GET(url):
  r = get(url)
  if r.status_code == 200:
    return r
  else: raise Exception('Error '+ str(r.status_code) + "! Most likely you entered an unsupported version. Try running the code again if you think that shouldn't have happened.")
def DOWNLOAD_FILE(url, path, file_name):
  # Download file into file_name thourgh url
  r = GET(url)
  LOG('\nDownloading ' + file_name)
  with open(path + '/' + file_name, 'wb') as f:
    f.write(r.content)

def COLABCONFIG(server_name):
  return f"{drive_path}/{server_name}/colabconfig.txt"
def COLABCONFIG_LOAD(server_name):
  if exists(COLABCONFIG(server_name)): return load(open(COLABCONFIG(server_name)))
  else:
    if exists(COLABCONFIG(server_name)): dump({"server_type": "generic"}, open(COLABCONFIG(server_name), 'w')) # using default, if config doesn't exist.
    raise Exception('Please checking whether you deleted your colabconfig file or not.')

SERVERCONFIG = f'{drive_path}/server_list.txt'
def SERVER_IN_USE(server_name):
  if exists(f'{drive_path}/{server_name}') and server_name != '': return server_name
  else:
    serverconfig = load(open(SERVERCONFIG))
    if serverconfig['server_in_use'] != '': return serverconfig['server_in_use']
    else:
      raise Exception('Please create a minecraft server first!')
# The jar file name
def JAR_LIST_RUN(colabconfig):
  return {'generic': 'server.jar', 'vanilla':'server.jar','snapshot': 'server.jar',
          'purpur' : 'server.jar', 'paper': 'server.jar', 'velocity' : 'server.jar',
          'fabric' : 'server.jar',
          'forge': 'minecraft_server.' + colabconfig['server_version'] + '.jar', # forge can't change the jar file
          'arclight' : 'server.jar', 'mohist': 'server.jar', 'banner': 'server.jar'}
#------------------------------------------------------- INSTALL SERVER.JAR -------------------------------------------------------------#
class Server:
  def __init__(self, server_name, server_type= '', version= ''):
    self.server_name = server_name
    self.server_type = server_type
    self.version = version
  def SERVERSJAR(self, all = False, jar = False):
    #Get the download URL (jar) AND return the detailed versions for each software (all)
    server_type = self.server_type
    version= self.version
    if all:
      Server_Jars_All = {
        'paper': 'https://api.papermc.io/v2/projects/paper', 'velocity': 'https://api.papermc.io/v2/projects/velocity',
        'purpur': 'https://api.purpurmc.org/v2/purpur',
        'mohist': 'https://mohistmc.com/api/v2/projects/mohist', 'banner': 'https://mohistmc.com/api/v2/projects/banner'
      }
      if server_type == 'vanilla' or server_type=='snapshot':
        rJSON = GET('https://launchermeta.mojang.com/mc/game/version_manifest.json').json()
        if server_type == 'vanilla': server_type = 'release'
        if version != 'vanilla - latest_version': server_version = [hit["id"] for hit in rJSON["versions"] if hit["type"] == server_type]
        else:
          return rJSON['latest']['release']
      elif server_type == 'paper' or  server_type == 'velocity' or server_type == 'purpur' or server_type == 'mohist' or server_type == 'banner':
        rJSON = GET(Server_Jars_All[server_type]).json()
        server_version = [hit for hit in rJSON["versions"]]
      elif server_type == 'fabric':
        rJSON = GET('https://meta.fabricmc.net/v2/versions/game').json()
        server_version = [hit['version'] for hit in rJSON["games"] if hit['stable'] == 'true']
      elif server_type == 'forge':
        rJSON = GET('https://files.minecraftforge.net/net/minecraftforge/forge/index.html')
        soup = BeautifulSoup(rJSON.content, "html.parser")
        server_version = [tag.text for tag in soup.find_all('a') if '.' in tag.text and '\n' not in tag.text]
      else:
        LOG('Before going deeper, please check out https://github.com/IzzelAliz/Arclight')
        rJSON = GET('https://files.hypoglycemia.icu/v1/files/arclight/minecraft').json()
        server_version  = [hit['name'] for hit in rJSON["files"]]
      return server_version
    elif jar == True and version != None:
      # RETURN DOWNLOAD URL
      if server_type == 'vanilla' or server_type=='snapshot':
        rJSON = GET('https://launchermeta.mojang.com/mc/game/version_manifest.json').json()
        if server_type == 'vanilla': server_type = 'release'
        for hit in rJSON["versions"]:
          if hit["type"] == server_type and hit['id'] == version:
            return GET(hit['url']).json()["downloads"]['server']['url']
      elif server_type == 'paper' or  server_type == 'velocity':
        build = GET(f'https://api.papermc.io/v2/projects/{server_type}/versions/{version}').json()["builds"][-1]
        jar_name = GET(f'https://api.papermc.io/v2/projects/{server_type}/versions/{version}/builds/{build}').json()["downloads"]["application"]["name"]
        return f'https://api.papermc.io/v2/projects/{server_type}/versions/{version}/builds/{build}/downloads/{jar_name}'
      elif server_type == 'purpur':
        build = GET(f'https://api.purpurmc.org/v2/purpur/{version}').json()["builds"]["latest"]
        return f'https://api.purpurmc.org/v2/purpur/{version}/{build}/download'
      elif server_type == 'mohist' or server_type == 'banner':
        return GET(f'https://mohistmc.com/api/v2/projects/{server_type}/{version}/builds').json()["builds"][-1]["url"]
      elif server_type == 'fabric':
        installerVersion = GET('https://meta.fabricmc.net/v2/versions/installer').json()[0]["version"]
        fabricVersion = GET(f'https://meta.fabricmc.net/v2/versions/loader/{version}').json()[0]["loader"]["version"]
        return "https://meta.fabricmc.net/v2/versions/loader/" + version + "/" + fabricVersion + "/" + installerVersion + "/server/jar"
      elif server_type == 'forge':
        rJSON = GET(f'https://files.minecraftforge.net/net/minecraftforge/forge/index_{version}.html')
        soup = BeautifulSoup(rJSON.content, "html.parser")
        tag =  soup.find('a', title="Installer"); tag = str(tag); tag = tag[tag.find('"') + 1 :]
        link = tag[:tag.find('"')]; link = link[link.find('=') + 1:]; link = link[link.find('=') + 1:]
        return link
      else:
        rJSON = GET(f'https://files.hypoglycemia.icu/v1/files/arclight/minecraft/{version}/loaders').json()
        LOG('Available type: '); print([hit['name'] for hit in rJSON['files']])
        build = input(' Type: '); choice = input('Stable(st) or Snapshot(sn): ')
        if choice == 'sn': choice = 'latest-snapshot';
        else: choice = "latest-stable";
        return f'https://files.hypoglycemia.icu/v1/files/arclight/minecraft/{version}/loaders/{build}/{choice}'
    else: raise Exception('Wrong given arguments')
  def Install_server(self, tunnel_service):
    # Auditing whether file is existed.
    if exists(f'{drive_path}/{self.server_name}') == False:
      # Create folder
      ! mkdir -p $drive_path/$self.server_name
      # Get version
      if self.version == '':
        # Print available version => get version (string variable)
        LOG(f'Latest versions/builds for {self.server_type}')
        server_version = self.SERVERSJAR(all = True)
        print('\n Aivailable version/builds:\n', server_version)
        version = input('Server versions: ')
        while version == '':
          LOG("Invalid versions")
          print('\n Aivailable version/builds:\n', server_version)
          version = input('Server version: ')
        self.version = version
      elif self.version == 'vanilla - latest_version': self.version = self.SERVERSJAR(all = True)
      # Load serverconfig
      serverconfig = load(open(SERVERCONFIG))
      serverconfig['server_list'] += [self.server_name]
      serverconfig['server_in_use'] = self.server_name
      if serverconfig['ngrok_proxy'] == {"authtoken": "", "region": ""} and tunnel_service == 'ngrok':
        LOG('Get your authtoken from https://dashboard.ngrok.com/get-started/your-authtoken')
        serverconfig['ngrok_proxy']['authtoken'] = input('Your authtoken: ')
        LOG('Available Regions:', ' ap - Asia/Pacific (Singapore)', ' au - Australia (Sydney)', ' eu - Europa (Frankfurt - Germany)', ' in - India (Mumbai)', ' jp - Japan (Tokyo)', ' sa - America (São Paulo - Brazil)', ' us - United States (Ohio)', sep='\n')
        serverconfig['ngrok_proxy']['region'] = input('Region: ')
      dump(serverconfig, open(SERVERCONFIG, 'w'))
      # Set up colabconfig
      colabconfig = {"server_type": self.server_type, "server_version": self.version, "tunnel_service" : tunnel_service}
      dump(colabconfig, open(COLABCONFIG(self.server_name),'w'))
      # Download jar file
      if self.server_type == 'forge': jarname = 'forge-installer.jar' # The jar file name (forge need a special process)
      else: jarname = JAR_LIST_RUN(colabconfig)[self.server_type]
      # Get jar file
      DOWNLOAD_FILE(url= self.SERVERSJAR(jar = True), path = f"{drive_path}/{self.server_name}", file_name= jarname)
      LOG('\nCompleted!')
    else: raise Exception('Lol, you have already installed this server file')
  def Delete_server(self):
    # Get default server
    self.server_name = SERVER_IN_USE(self.server_name)
    serverconfig = load(open(SERVERCONFIG))
    if serverconfig['server_list'] == []: raise Exception("You haven't installed yet.")
    else:
      # Auditing whether file is existed.
      if exists(f'{drive_path}/{self.server_name}') == False: raise Exception("You haven't installed yet.")
      elif self.server_name == '':
        if serverconfig['server_in_use'] != '': return serverconfig['server_in_use']
      LOG(f'Deleting {self.server_name}...')
      # Delete folder without noticable
      !rm -rf "{drive_path}/{server_name}" &>/dev/null&
      # Remove the folder name in server config txt files
      serverconfig['server_list'].remove(self.server_name)
      if serverconfig['server_in_use'] == self.server_name:
        try: serverconfig['server_in_use']= serverconfig['server_list'][0]
        except: serverconfig['server_in_use'] = ''
      dump(serverconfig, open(SERVERCONFIG, 'w'))
      LOG('Completed')
  @staticmethod
  def Choose_server():
    serverconfig = load(open(SERVERCONFIG))
    if serverconfig['server_list'] == []: raise Exception(' Creating your minecraft server before choosing')
    else:
      LOG('\nAivailable server:\n')
      for server in serverconfig['server_list']:
        LOG(server)
      serverconfig['server_in_use'] = input('\nYour minecraft server: ')
      dump(serverconfig, open(SERVERCONFIG, 'w'))
      LOG('Completed')

class Cmd:
  def __init__(self, server_name):
    self.server_name = SERVER_IN_USE(server_name)
    self.colabconfig = COLABCONFIG_LOAD(self.server_name)
    self.serverconfig = load(open(SERVERCONFIG))
    self._type = self.colabconfig["server_type"]
    self.version = self.colabconfig["server_version"]
    self.tunnel_service = self.colabconfig["tunnel_service"]
  def INSTALL_JAVA(self):
    # Adoptopen jdk -- optional
    # !wget -qO - https://adoptopenjdk.jfrog.io/adoptopenjdk/api/gpg/key/public | sudo apt-key add -
    # !sudo add-apt-repository --yes https://adoptopenjdk.jfrog.io/adoptopenjdk/deb/ &>/dev/null || echo "Failed to add repo. Still can be ignored if openjdk gets installed."

    # Java 8 is required to run Minecraft versions 1.12 through 1.17. Java 17 is required to run Minecraft version 1.18 and up.
    if self.version > "1.17":
      ! sudo apt-get -y install openjdk-16-jre-headless  > /dev/null &&echo 'Openjdk17 is working correctly, you are good to go.' || "Openjdk doesn't seem to be installed or isn't working, falling back to java 11\nYou might experience reduced performance. Minecraft 1.18 and above might fail to launch."
      environ["JAVA_HOME"] = "/usr/lib/jvm/java-16-openjdk-amd64"
      ! update-alternatives --set java /usr/lib/jvm/java-16-openjdk-amd64/bin/java
    else:
      ! sudo apt-get -y install openjdk-8-jre-headless  > /dev/null && echo 'Openjdk8 is working correctly, you are good to go.' || echo "Openjdk doesn't seem to be installed or isn't working, falling back to java 11\nYou might experience reduced performance. Minecraft 1.18 and above might fail to launch."
      environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
      ! update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
  def CONNECT_NGROK(self, port, type_ , proxy):
    # Get serverconfig['ngrok_proxy'] : dict includes (authtoken, region)
    token = proxy['authtoken']
    !ngrok authtoken $token
    region = proxy['region']
    conf.get_default().region = region
    url = ngrok.connect (port, type_)
    if type_ == 'tcp': return 'Your server address is ' + ((str(url).split('"')[1::2])[0]).replace('tcp://', '')
    else: return url
  def EXIT_NGROK(self):
    # Exiting ngrok tunnel helps improve the performance
    tunnels = ngrok.get_tunnels()
    for tunnel in tunnels: ngrok.disconnect(tunnel.public_url)
    ngrok.kill()
    LOG("Ngrok closed.")
  def RUNCOMMAND(self, hide = False):
    # Get all the improving java arguments
    java_ver = !java -version 2>&1 | awk -F[\"\.] -v OFS=. 'NR==1{print $2}'
    args = " -Xms8G -Xmx8G "
    if self._type == "paper" or self._type == 'purpur':
      # Improving paper cilent (purpur is an alternative).
      # For more detailed: https://docs.papermc.io/paper/aikars-flags
      args += '-XX:+UseG1GC -XX:+ParallelRefProcEnabled -XX:MaxGCPauseMillis=200 -XX:+UnlockExperimentalVMOptions -XX:+DisableExplicitGC -XX:+AlwaysPreTouch -XX:G1NewSizePercent=30 -XX:G1MaxNewSizePercent=40 -XX:G1HeapRegionSize=8M -XX:G1ReservePercent=20 -XX:G1HeapWastePercent=5 -XX:G1MixedGCCountTarget=4 -XX:InitiatingHeapOccupancyPercent=15 -XX:G1MixedGCLiveThresholdPercent=90 -XX:G1RSetUpdatingPauseTimePercent=5 -XX:SurvivorRatio=32 -XX:+PerfDisableSharedMem -XX:MaxTenuringThreshold=1 -Dusing.aikars.flags=https://mcflags.emc.gs -Daikars.new.flags=true '
    elif self._type == "velocity":
      # Java argument for velocity server
      # For more detailed: https://docs.papermc.io/velocity/tuning
      args += '-XX:+UseG1GC -XX:G1HeapRegionSize=4M -XX:+UnlockExperimentalVMOptions -XX:+ParallelRefProcEnabled -XX:+AlwaysPreTouch -XX:MaxInlineLevel=15'
    # GC_LOGGING
    if java_ver[0] == "1": args += '-Xloggc:gc.log -verbose:gc -XX:+PrintGCDetails -XX:+PrintGCDateStamps -XX:+PrintGCTimeStamps -XX:+UseGCLogFileRotation -XX:NumberOfGCLogFiles=5 -XX:GCLogFileSize=1M'
    # else: args += '-Xlog:gc*:logs/gc.log:time,uptime:filecount=5,filesize=1M' # GC_Logging may not worked in java 11 and above

    # Forge need to open and run it before starting server. => Using command instead.
    if self._type == 'forge':
      !java -jar forge-installer.jar --installServer
      LOG('\nServer starting...')
    jar_name = JAR_LIST_RUN(self.colabconfig)[self._type]

    if hide == True and self._type != 'mohist' and self._type != 'banner':
        # Install needed file: eula.txt and so on.
        !java -server $args -jar $jar_name nogui >/dev/null && wait&
    else:
      # Starting tunneling and run java file.
      LOG(f'Starting server ({self.tunnel_service})...')
      LOG('Stop server => /stop')
      if self.tunnel_service == "ngrok":
        LOG(self.CONNECT_NGROK(type_= 'tcp', port= 25565, proxy = self.serverconfig['ngrok_proxy']))
        !java -server $args -jar $jar_name nogui
      elif self.tunnel_service == "playit":
        # Download playit
        ! command -v playit || curl -SsL https://playit-cloud.github.io/ppa/key.gpg | gpg --dearmor | sudo tee /etc/apt/trusted.gpg.d/playit.gpg &>/dev/null && echo "deb [signed-by=/etc/apt/trusted.gpg.d/playit.gpg] https://playit-cloud.github.io/ppa/data ./" | sudo tee /etc/apt/sources.list.d/playit-cloud.list && sudo apt -qq update && sudo apt install playit &>/dev/null && echo "Playit.gg installed" || echo "Failed to install playit"
        # Run server
        ! playit & java -server $args -jar $jar_name nogui
      else:
        # Download argo
        if exists(f'{drive_path}/{self.server_name}/cloudflared-linux-amd64') == False:
          ! curl https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 > cloudflared-linux-amd64
          ! chmod +x cloudflared-linux-amd64
        # Run server
        !./cloudflared-linux-amd64 tunnel --url tcp://127.0.0.1:25565 && echo 'Your free tunnel has started!' && java -server $args -jar $jar_name nogui
  def Run_server(self):
    # Cd to server path
    %cd $drive_path/$self.server_name
    # Install java acording to the version of server.jar
    self.INSTALL_JAVA()
    # Install necessary files
    if exists(f'{drive_path}/{self.server_name}/eula.txt') == False:
      LOG("\n[Note: Downloading essentials files takes approximately 10-15 minutes. Please wait for a little bit.]")
      self.RUNCOMMAND(hide = True)
      # Accept EULA
      LOG('\nAccepting Eula.')
      !sed -i 's/eula=false/eula=true/g' eula.txt
    # See what changes in the folder
    LOG("\nFiles:")
    !ls -a
    # Run minecraft server
    self.RUNCOMMAND()
    # Stop minecraft server => Exit tunnel => Cd to home
    LOG("Finalized server.")
    if self.colabconfig['tunnel_service'] == 'ngrok': self.EXIT_NGROK()
    %cd $drive_path

class Download_:
  def __init__(self, server_name, software, url, search_name, categories, versions, proj_types, index):
    self.server_name = SERVER_IN_USE(server_name)
    self.colabconfig = COLABCONFIG_LOAD(server_name)
    self.search_name = search_name.lower()
    self.software = software
    self.url = url
    if versions == 'default': versions = colabconfig["server_version"]
    if categories == 'default': categories = colabconfig['server_type']
    if proj_types == 'default' :
      if 'fabric' in categories or 'forge' in categories : proj_types = 'mods'
      elif 'paper' in categories or 'purpur' in categories : proj_types = 'plugins'
      elif 'arclight' in categories or 'mohist'  in categories or 'banner': proj_types = input('Project type (mods or plugins or modpacks) : '); categories = 'none'
      else: proj_types = ''; categories = 'none'
    elif proj_types == 'none': proj_types = 'tmp'
    if 'arclight' in categories or 'mohist'  in categories or 'banner' or 'vanilla' in categories or 'snapshot' in categories: categories = 'none';
    elif 'paper' in categories or 'purpur' in categories:
      if proj_types == 'mods': categories = 'none';
    elif 'fabric' in categories or 'forge' in categories:
      if proj_types == 'plugins': categories = 'none'
    self.categories = categories
    self.versions = versions
    self.proj_types = proj_types
    self.index = index
    self.path = f'{drive_path}/{self.server_name}/{self.proj_types}'
  def FACETS(self):
    # Get all the syntax
    if software == 'Modrinth':
      facets = "["
      if self.categories != 'none': facets += '["categories:' + self.categories + '"]';
      if facets != '[' and self.versions != 'none': facets += "," + '["versions:' + self.versions +'"]'
      elif self.versions != 'none': facets += '["versions:' + self.versions + '"]';
      if self.proj_types == 'mods': proj_types = 'mod'
      elif self.proj_types == 'plugins': proj_types = 'plugin'
      if facets != '[' and proj_types != 'tmp': facets += "," + '["project_type:' + proj_types + '"]'
      elif self.proj_types != 'tmp': facets += '["project_type:' + proj_types + '"]'
      facets += "]"; facetsInURL = "";
      if facets != "[]": facetsInURL += f'&facets={facets}'
      if self.index != "none": facetsInURL += f'&index={self.index}'
      return facetsInURL
    else:
      if self.categories != 'none':
        if self.categories == 'fabric': categories = 4 #Fabric
        elif self.categories == 'forge': categories = 1 #Forge
        elif self.categories == 'quilt': categories = 5 # quilt
        categories = f"&modLoaderType={categories}"
      if self.index != "none":
        if self.index == "relevance": index = 1 # Featured
        elif self.index == "downloads": index = 6 #TotalDownloads
        elif self.index == "follows": index = 2 #Popularity
        elif self.index == "newest": index = 11 #ReleasedDate
        elif self.index == "updated": index = 3 #LastUpdated
        index = f"&sortField={index}"
      else: index = ''
      if self.versions != "none": versions = f"&gameVersion={versions}"
      else: versions = ""
      return categories + versions + index
  def SEARCH(self):
    project = {}
    LOG(f'\nSearching for the related of {self.search_name} ...\n')
    facetsInURL = self.FACETS()
    if self.software == "Modrinth":
      # Get syntax and get data
      rJSON = GET(f'https://api.modrinth.com/v2/search?query={self.search_name}{facetsInURL}').json()
      # Get the list of all relevant project
      for hit in rJSON['hits']:
        # Auditing if it for server or not.
        if hit['server_side'] == 'optional' or hit['server_side'] == 'required':
          # Get a full list title : description
          print(hit['slug'], " : ", hit['description'])
          project[hit['slug']] = hit['project_id']
    elif software == 'Curseforge':
      LOG(f"Curseforge doesn't have the exact searching key for cilent-side or server-side => you may get errors when running this {self.proj_types}")
      # Because I haven't found any corresponded of project types in the search engine of Curseforge, I don't use it for searching.
      # The gameid of Minecraft is 432
      rJSON = GET(f"https://api.curse.tools/v1/cf/mods/search?gameId=432&searchFilter={self.search_name}{facetsInURL}").json()
      # Get the list of all relevant project
      for hit in rJSON["data"]:
        # Get a full list name: summary
        print(hit["name"], ' : ', hit["summary"])
        project[hit['name']] = str(hit["id"])
    # Checking whether your search name wrong or not. If yes => Get the name of project => Get project id
    project_names =''
    if project == {}:
      raise Exception(f"\nSomething went wrong. Please check your search name.")
    else:
      LOG('\nType the project_name you want to download')
      project_names= input('Project_name: ')
      while project_names not in project:
        LOG('\nWrong project_names please type aigain. If you want to quit, type "None".')
        project_names= input('\nProject_name: ')
        if project_names == 'None': raise Exception('Stopping...')
    return [project, project_names]
  def MODPACK(self, file_name):
    # unzip modpack
    !unzip file_name -d file_name.split('.')[0] &>/dev/null&
    # Copy the directory in orverrides and paste it (overrides includes the config files of the developer)
    !cp -r $file_name/overrides $drive_path/$self.server_name
    # Each pahe give th difference file json. The file json give full details
    # (Modrinth: modrinth.index.json- Download link, Curseforge: manifest.json - fileID, projectID) for mods which is included in modpack.
    if software == 'Modrinth':
      manifest = load(open(f"{self.path}/modrinth.index.json"))["files"]
      for files in manifest:
        path_ = files["path"].split("/")[0]
        file_name_ = files["path"].split("/")[1]
        DOWNLOAD_FILE(url = files["downloads"], path = f'{drive_path}/{self.server_name}/{path_}',file_name = file_name_)
    else:
      manifest = load(open(f"{self.path}/manifest.json"))["files"]
      for files in manifest:
        project_id = files["projectID"]
        file_ID = files["fileID"]
        rJSON = GET(f'https://api.curse.tools/v1/cf/mods/{project_id}/files/{fileId}').json()["data"]
        file_name_ = rJSON["displayName"]
        DOWNLOAD_FILE(url = files["downloads"], path = f'{drive_path}/{self.server_name}/mods', file_name = file_name_)
  def Install_(self):
    LOG(f'Acessing: {self.server_name}')
    if self.url != '':
      # Find file_name in download url
      filename = input('File name (optional) : ')
      if filename == '':
        filename = url[url.find("/") + 1:]
        while filename.find("/")!= -1: filename = filename[filename.find("/") + 1:]
      # Download file
      DOWNLOAD_FILE(url= self.url, path = self.path, file_name= file_name)
      if self.proj_types == 'modpack': self.MODPACK(file_name= file_name);
      LOG('\nCompleted')
    else:
      a = self.SEARCH()
      project = a[0]; project_names = a[1]; check = False;
      project_id = project[project_names]
      if exists(self.path) == False:
        !mkdir $self.path
      if software == 'Modrinth': rJSON = GET(f'https://api.modrinth.com/v2/project/{project_id}/version').json()
      else: rJSON = GET(f'https://api.curse.tools/v1/cf/mods/{project_id}').json()['data']['latestFilesIndexes'];
      for data in rJSON:
        if software == 'Curseforge': gameversions= data["gameVersion"]
        else: gameversions= data["game_versions"]
        if versions in gameversions:
          if software == 'Curseforge': files = data, fileId = data['fileId']; url = f'https://www.curseforge.com/api/v1/mods/{project_id}/files/{fileId}/download'
          else: files = data['files'][0]; url = files['url']
          DOWNLOAD_FILE(url= self.url, path = self.path, file_name= files['filename'])
          check = True
        if check == True and self.proj_types == 'modpack':
          self.MODPACK(file_name= file_name); print('\nCompleted.'); break
        elif check: print('\nCompleted.'); break
      if check == False: raise Exception(f"It seems that {self.software} doesn't support this {self.proj_types}")

#--------------------------------------------------- Ulities ----------------------------------------------------------#
class Ulities:
  @staticmethod
  def Custom_props(server_name, server_icon, motd):
    # Default server
    server_name = SERVER_IN_USE(server_name)
    if exists(f"{drive_path}/{server_name}/server.properties") == False: raise Exception(' Creating your minecraft server before editing properties')
    else:
      LOG('Server:  ' + server_name)
      # Add server icon file
      if server_icon != '': DOWNLOAD_FILE( url = server_icon, path = f'{drive_path}/{server_name}', file_name = 'server_icon.png')
      # Editing MOTD
      if motd != '':
        server_properties = Properties()
        with open(f"{drive_path}/{server_name}/server.properties", "rb") as f:
            server_properties.load(f, "utf-8")
        server_properties["motd"] = motd
        with open(f"{drive_path}/{server_name}/server.properties", "wb") as f:
            server_properties.store(f, encoding="utf-8")
      LOG('Completed')
  @staticmethod
  def Serverproperties(
      server_name, Slots, Gamemode, Difficulty, Cracked, PVP, Command_block, Fly,
      Animals, Monsters, Villagers, Nether, Force_gamemode, Spawn_protection):
    server_name = SERVER_IN_USE(server_name)
    if exists(f"{drive_path}/{server_name}/server.properties") == False: raise Exception(' Creating your minecraft server before editing properties')
    else:
      server_properties = Properties()
      with open(f"{drive_path}/{server_name}/server.properties", "rb") as f:
          server_properties.load(f, "utf-8")
      # Configuring
      server_properties["max-players"] = str(Slots)
      server_properties["gamemode"] = Gamemode
      server_properties["difficulty"] = Difficulty
      dict_ = {'pvp': PVP, 'enable-command-block': Command_block, 'allow-flight': Fly, 'spawn-animals': Animals, 'spawn-monsters': Monsters,
              'spawn-npcs': Villagers, 'allow-nether': Nether, 'force-gamemode': Force_gamemode, 'spawn-protection': Spawn_protection, "online-mode" : Cracked}
      for keys, value in dict.items(): server_properties[keys] = str(value).lower()
      # Saving
      with open(f"{drive_path}/{server_name}/server.properties", "wb") as f:
          server_properties.store(f, encoding="utf-8")
      LOG('Completed')
  @staticmethod
  def Software(server_name, server_type, version = ''):
    serverconfig = load(open(SERVERCONFIG))
    colabconfig = load(open(COLABCONFIG(server_name)))
    if server_name == '': server_name = serverconfig['server_in_use']
    Delete_server(server_name)
    sleep(20)
    Install_server(colabconfig['tunnel_service'], serverconfig['ngrok_proxy']['authtoken'], serverconfig['ngrok_proxy']['region'], server_name, server_type)
  @staticmethod
  def Backup_files(file_name = '', file_backup = '', server_name= '', server_backup = '', path = '/content/drive/MyDrive/minecraft'):
    # Get the default server
    server_name = SERVER_IN_USE(server_name)
    # Settings path
    if file_name != '':
      path1 = path + f'/{server_name}/' + file_name
      if server_backup != '':
        if exists(f'{drive_path}/{server_backup}') == False:
          !mkdir '{drive_path}/{server_backup}'
      if file_backup != '' and server_backup != '': path2 = path + f'/{server_backup}/' + file_backup
      elif file_backup != '' and server_backup == '': path2 = path + f'/{server_name}/' + file_backup
      elif file_backup == '' and server_backup != '':  path2 = path + f'/{server_backup}/' + '(back-up)'
      else: path2 = path + f'/{server_name}/' + '(back-up)'
    else:
      path1 = path + f'/{server_name}'
      if server_backup != '': path2 = path + f'/{server_backup}'
      else: path2 = path + f'/{server_name}' + '(back-up)'
    if exists(path1) == False: raise Exception(' Creating your minecraft server before backing up files')
    if exists(path2) == True: raise Exception(' Back up path exists')
    # Zipping
    !zip -r $path2.zip $path1 &>dev/null && echo "Zipping done!" || echo "Zipping faled."

#------------------------------------------------------------------------------------------------------------------------------------#
!sudo apt update &>/dev/null &&echo 'apt cache successfully updated' || echo "apt cache update failed, you might receive stale packages"
!sudo apt upgrade &>/dev/null &&echo 'apt cache successfully upgraded' || echo "apt cache upgrade failed, you might receive stale packages"
if exists(drive_path) == False:
  !mkdir -p $drive_path &>/dev/null&
if exists(SERVERCONFIG) == False:
  dump({"server_list": [], "server_in_use": "", "ngrok_proxy" : {'authtoken' : '', "region" : ''}}, open(SERVERCONFIG, 'w'))
%cd $drive_path
LOG('Completed')

---

# 🔥 **Starting**
---


In [ ]:
# @markdown ####**Choose your server file name**
server_name = "minecraft" # @param {type:"string"}

Server(

  #------------------------------#
    server_name = server_name,
    server_type = "vanilla", # server type
    version = 'vanilla - latest_version',

).Install_server(

    # @markdown ####You can either use [ngrok](https://ngrok.com/), [playit.gg](https://playit.gg/) or [cloudflare's argo](https://www.cloudflare.com/). Ngrok is easy to setup and doesn't requires anything to be installed by the clients but it can often be quite unreliable. Argo doesn't have such limitations but requires a bit more work. Playit has provided limited versions but with a static domain.
    # Standard turnel service ( Aivailable turnel: ngrok, argo, playit)
    tunnel_service = 'ngrok' #@param ['ngrok', 'argo', 'playit']

)

In [ ]:
# @markdown ####**Choosing minecraft server**

Server.Choose_server()

In [ ]:
# @markdown ####**Delete server**
Server(server_name = 'testing').Delete_server()

-----------------------------------------------------

# ▶ **or** 🛑  **Console**
---
The main console for your minecraft server

- **[Ngrok](https://ngrok.com)**
  + Follow the prompts.
  + The IP will change whenever you restart the server.
- **[Cloudflare's argo](https://www.cloudflare.com/)** :
    - If the 'Your free tunnel has started!' notification appears => Done.
    - Access to your server:
    1. Download [Cloudflared client](https://github.com/cloudflare/cloudflared/releases/).
    2. Launch the binary with `<your Cloudflare file name> access tcp --hostname <tunnel_address> --url 127.0.0.1:25565` (note: tunnel_address is your address which has been set on your Cloudflare)
    4. Finally, connect to `127.0.0.1:25565` from the minecraft client which is located in that machine.
- **[PlayIt](https://playit.gg/)**: follow the prompts.

In [ ]:
Cmd(
    #------------------------------#
    # Leaving emty string   -> set to default files
    # @markdown
    server_name = ''
    #------------------------------#
).Run_server()

---

# ⚓ **Options**
---


In [ ]:
# @markdown ###**Server_custom**

# @markdown ####Server icon: **upload image** and get **direct link** from [[ Image ]](https://postimages.org/)
Server_icon = '' # @param {type: 'string'}
# @markdown ####[[ MOTD create **tool** ]](https://mctools.org/motd-creator)
MOTD = '' # @param {type: 'string'}


Ulities.Custom_props(
    #------------------------------#
    server_name = '', # Customized server file
    #------------------------------#
    server_icon = Server_icon,
    motd = MOTD
    #------------------------------#

)

In [ ]:
# @markdown ###**Server properties**

Slots = 20 # @param {type:"slider", min:0, max:100, step:1}
Gamemode = "survival" # @param ["survival", "creative", "adventure", "spectator"]
Difficulty = "easy" # @param ["peaceful", "easy", "normal", "hard"]
Cracked = True # @param {type:"boolean"}
PVP = True # @param {type:"boolean"}
Command_block = True # @param {type:"boolean"}
Fly = True # @param {type:"boolean"}
Animals = True # @param {type:"boolean"}
Monsters = True # @param {type:"boolean"}
Villagers = True # @param {type:"boolean"}
Nether = True # @param {type:"boolean"}
Force_gamemode = False # @param {type:"boolean"}
Spawn_protection = 16 # @param {type:"slider", min:0, max:100, step:1}



Ulities.Serverproperties(
    server_name = '',

    Slots,
    Gamemode,
    Difficulty,
    Cracked,
    PVP,
    Command_block,
    Fly,
    Animals,
    Monsters,
    Villagers,
    Nether,
    Force_gamemode,
    Spawn_protection

)

---

# 📎  **Log**
---


In [ ]:
# @markdown **Shows latest log of your minecraft server**
# Getting server_folder

server_name = SERVER_IN_USE()
if os.path.exists(f'/content/drive/My Drive/minecraft/{server_folder}/logs/latest.log'):
  print(f'Server {server_name}')
  with open(f'/content/drive/My Drive/minecraft/{server_folder}/logs/latest.log', 'r') as f:
    print(f.read())
else: EROR("Error: File didn't exists")

---


# 📰  **Software**
---
Change the software (vanilla) to others platforms

---


+ Please checking whether your minecraft server is in Gdrive or not.


In [ ]:
Download_.Software(

    # Default server
    server_name = '',

    # Server_version
    version = '', # Customizable version

    # @markdown ####*Choosing server type*
    server_type = "vanilla" # @param ["vanilla", "snapshot", "paper", "purpur", "forge", "fabric", 'velocity', 'arclight','mohist', 'banner']

)

---

# 🎈  **Plugins, mods**

####Download modpack/mod/plugin from [curseforge](https://www.curseforge.com/Minecraft) and [modrinth](https://modrinth.com/)
---


In [ ]:
#@markdown ### **Your search name**

search_name = 'essentials' # @param {type: 'string'}

# @markdown ####Choose the place to download mod/modpack/plugin
software = 'Modrinth' # @param ["Curseforge", "Modrinth"]

# @markdown ###**Details:**
# @markdown
# @markdown #### (none -> don't search (version -- doesn't wellcoming to do this), default -> set up according to colabconfig file)
categories = 'default'   #@param ['none', 'default', 'vanilla', 'fabric', 'forge', 'paper', 'purpur']
versions = "default" # @param ["default"] {allow-input: true}
project_types = 'plugins' # @param ['none', 'default', 'mods', 'plugins', 'modpacks']
index = 'none' #@param ['none', 'relevance', 'downloads', 'follows', 'newest', 'updated']

Download_(
    #------------------------------#
    server_name = '',
    software= software,
    url= '', # Customized urlmodpack (just from curseforge or modpack)
    #------------------------------#
    search_name= search_name.lower(),
    categories = categories.lower(),
    versions= versions.lower(),
    proj_types= project_types.lower(),
    index= index.lower()
).Install_()

---

# **📁 File Management**
---


In [ ]:
# @markdown Back up server or file?
choice = 'server' # @param ['server', 'file']
file_name = ''; file_backup = '';  server_backup = ''
if choice == 'server':
  server_backup = input('File back up name: ')
else:
  file_name = input('File name')
  file_backup = input('File back up name: ')
Ulities.Backup_files(
    #------------------------------#
    file_name = file_name,
    file_backup = file_backup,
    server_name= SERVER_IN_USE(server_name= ''),
    server_backup = server_backup,
    path = '/content/drive/MyDrive/minecraft'
    #------------------------------#
)